In [1]:
# def main(debug=False):
#     num_rows = 10000 if debug else None
#     with timer("train & test"):
#         df = train_test(num_rows)
#     with timer("historical transactions"):
#         df = pd.merge(df, historical_transactions(num_rows), on='card_id', how='outer')
#     with timer("new merchants"):
#         df = pd.merge(df, new_merchant_transactions(num_rows), on='card_id', how='outer')
#     with timer("additional features"):
#         df = additional_features(df)
#     with timer("split train & test"):
#         train_df = df[df['target'].notnull()]
#         test_df = df[df['target'].isnull()]
#         del df
#         gc.collect()
#     with timer("Run LightGBM with kfold"): 
#         kfold_lightgbm(train_df, test_df, num_folds=11, stratified=False, debug=debug)

# if __name__ == "__main__":
#     submission_file_name = "submission.csv"
#     with timer("Full model run"):
#         main(debug=False)

In [17]:
from contextlib import contextmanager
import pandas as pd
import numpy as np
import time

In [3]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

### Preprocessing train & test

In [12]:
# load csv
num_rows=None
train_df = pd.read_csv('../data/train.csv', index_col=['card_id'], nrows=num_rows)
test_df = pd.read_csv('../data/test.csv', index_col=['card_id'], nrows=num_rows)

print("Train samples: {}, test samples: {}".format(len(train_df), len(test_df)))
display(train_df.head())
display(test_df.head())
train_df.describe().T

Train samples: 201917, test samples: 123623


,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283
C_ID_3d0044924f,2017-01,4,1,0,0.392913
C_ID_d639edf6cd,2016-08,2,2,0,0.688056
C_ID_186d6a6901,2017-09,4,3,0,0.142495
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749


,first_active_month,feature_1,feature_2,feature_3
card_id,,,,
C_ID_0ab67a22ab,2017-04,3,3,1
C_ID_130fd0cbdd,2017-01,2,3,0
C_ID_b709037bc5,2017-08,5,1,1
C_ID_d27d835a9f,2017-12,2,1,0
C_ID_2b5e3df5c2,2015-12,5,1,1


,count,mean,std,min,25%,50%,75%,max
feature_1,201917.0,3.105311,1.186160,1.000000,2.00000,3.000000,4.000000,5.000000
feature_2,201917.0,1.745410,0.751362,1.000000,1.00000,2.000000,2.000000,3.000000
feature_3,201917.0,0.565569,0.495683,0.000000,0.00000,1.000000,1.000000,1.000000
target,201917.0,-0.393636,3.850500,-33.219281,-0.88311,-0.023437,0.765453,17.965068


In [16]:
# outlier
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1

In [18]:
# set target as nan
test_df['target'] = np.nan

In [22]:
# append train + test vertically ---> df
df = pd.concat([train_df, test_df])
df.shape

(325540, 6)